# SELENIUM INSTALLATION

In [31]:
"""
This is the Selenium installation
"""

!pip install selenium

!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager
"""
Just breaking it up
"""
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

"""
More additions for Selenium
"""

options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(30)


tiki_url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?page=1&src=c.1789.hamburger_menu_fly_out_banner'
driver.get(tiki_url)

html_data = driver.page_source   # after driver.get() is done, you can get back HTML string by using .page_source
driver.close() # close the browser after getting what you want


Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (89.0.4389.90-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


# SCRAPING DATA FROM TIKI - ALL AVAILABLE PAGES OF PHONE AND TABLE CATEGORIES

In [2]:
def tiki_scrape():
  from bs4 import BeautifulSoup
  def selenium_scrap(tiki_url):
    driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above  
    driver.implicitly_wait(30)                                       # We let selenium to wait for 30 seconds for all javascript script done before return the result of HTML                       
    driver.get(tiki_url)                                             # Open the browser again to get web page
    html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
    driver.close()                                                   # Close the driver after retrieving the web page
                                          
    soup = BeautifulSoup(html_data, 'html.parser')                   # do your beautifulsoup business like the usual
    products = soup.find_all('a', {'class':'product-item'})
    return products

  all_pages =[]
  for i in range (1,1000):
    products = selenium_scrap('https://tiki.vn/dien-thoai-may-tinh-bang/c1789?page='+str(i)+'&src=c.1789.hamburger_menu_fly_out_banner')
    if len(products)== 0:
      break
    else:
      for n in range (len(products)):
        item = products[n]
        all_pages.append(item)

  return all_pages

In [3]:
raw_data = tiki_scrape()

In [4]:
raw_data[1]

<a class="product-item" data-view-id="product_list_item" data-view-index="1" href="/dien-thoai-oppo-a12-3gb-32gb-hang-chinh-hang-xanh-duong-p53827214.html?src=ss-organic"><span class="style__StyledItem-sc-18svp8n-0 bHrHjw"><div><div class="thumbnail"><div class="badge-top"></div><img alt="Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng" src="https://salt.tikicdn.com/cache/280x280/ts/product/aa/f2/95/c0c73883ea383b2eedbeda927ab615dd.jpg"/></div><div class="style__OptionColor-sc-18svp8n-2 FzDve option-color"><div class="Slide__Root-sc-1bhg4sj-0 cIzQMm" style="width: 144px; margin: 16px auto 12px 0px;"><div class="Slide__Slider-sc-1bhg4sj-1 IZyLq"><div class="Slide__Frame-sc-1bhg4sj-2 hwwfcE"><div class="Slide__Item-sc-1bhg4sj-3 hMoQqt" style="border: 1px solid rgb(253, 130, 10);" width="25"><img class="style__BlockImg-sc-18svp8n-5 kmGwFP" src="https://salt.tikicdn.com/cache/64x64/ts/product/aa/f2/95/c0c73883ea383b2eedbeda927ab615dd.jpg"/></div><div class="Slide__Item-sc-1bhg4sj-3 hMoQqt

#EXTRACTING DATA FROM SCRAPING

In [84]:


def refund(a):
  """
  Rẻ hơn hoàn tiền
  To find out product which allows to get refund if customer could find the product cheaper anywhere else. False: Option not applicable
  """
  if len(raw_data[a].find('div',{'class':'badge-under-price'})) == 1: 
      return (True)
  else: 
      return (False)

def is_Now(b):
  """
  To find if the product has "Tiki now" option (to be delivered within the day):
  False: Tiki_now not applicable
  """
  if len(raw_data[b].find('div',{'class':'badge-service'})) == 1:
      return True
  else: 
      return False
 

def data_extract():
  data = []
  for n in range(len(raw_data)):
    d = {
        "product_ID": "",
        "product_title" : "",
        "price": "",
        "discount": "",
        "freeship": "",
        "product_img_url":"",
        "product_pg_url":"",
        "tiki_now":"",
        "installment_accepted":"",
        "total_reviews": "",
        "ratings_avg": "",
        "cheaper_getRefund":"","free_gift": ""
    }
    try:
        d["product_ID"] = ((re.search("(p\d+).html",raw_data[n]['href'])).group()).strip('.html')
        d["product_title"] = raw_data[n].find('div', {'class':'name'}).text
        
        #Product price
        d["price"] = int(raw_data[n].find('div', {'class':'price-discount__price'}).text.strip('₫').replace('.',''))



        #Discount 
        if raw_data[n].find('div', {'class':'price-discount__discount'}) is None:
            d["discount"] = ""
        else:
            d["discount"] = raw_data[n].find('div', {'class':'price-discount__discount'}).text


        #Freeship and product image: These 2 are under class "badge on top", some products have "Freeship" option >> first image is "Freeship icon", then product image. 
        #However many products dont have freeship option and product image now in the 1st place. 
        if raw_data[n].find('div', {'class':'thumbnail'}).text == '':
            d["freeship"] = ''
            d["product_img_url"] = raw_data[n].find('img').get('src')
        else:
            d["freeship"] = raw_data[n].find('div', {'class':'thumbnail'}).text
            d["product_img_url"] = raw_data[n].find_all('img')[1].get('src') 
    

        #Product URL:
        d["product_pg_url"] = 'tiki.vn'+raw_data[n]['href']


        #Tiki_now option:    
        d["tiki_now"] = is_Now(n)


        #Installment option 
        d["installment_accepted"] = raw_data[n].find('div', {'class':'badge-benefits'}).text# Shoots out result in Vietnamese which I think is fine.
        

        #Number of reviews
        if raw_data[n].find('div', {'class':'review'}) is None:
            d["total_reviews"] = 0
        else:
            d["total_reviews"] = int(raw_data[n].find('div', {'class':'review'}).text.strip('()'))


        #Ratings:
        if raw_data[n].find('div', {'class':'rating__average'}) is None:
            d["ratings_avg"] = 0
        else:
            d["ratings_avg"] = (float(raw_data[n].find('div', {'class':'rating__average'}).get("style").strip('width:').strip('%;')))/20


        #Rẻ hơn hoàn tiền    
        d["cheaper_getRefund"] = refund(n)

        data.append(d)
    except:
        print ("Error found!")
  return data

In [85]:
data = data_extract()

In [86]:
data

[{'cheaper_getRefund': True,
  'discount': '-26%',
  'free_gift': '',
  'freeship': '',
  'installment_accepted': 'Trả góp',
  'price': 4790000,
  'product_ID': 'p58259141',
  'product_img_url': 'https://salt.tikicdn.com/cache/280x280/ts/product/50/86/f1/8c08d595e64661d3d0d9438ca2194de3.jpg',
  'product_pg_url': 'tiki.vn//tka.tiki.vn/pixel?data=djAwMem5AAIaFuR4y_VY1jBGwEJ4dk_vpDBv7-ytE7UFRbJEwMzBE3FBlZyjAJbf-nLQRYkykU1kyqACq-fH1T32AA7Yzza_OdyMtT7PfnS9LyATLjskQjv7M8MHspv7Pg1KftcFOBTaZBf9hjxv_p0GLHmUmEeiRtcaNNDeKhZtipo2ss54qJv7GYEu8UNsXdi4JJEyhZuLmFDKL5Iv8b2Qfe1n0FgwvrbC4xh9-5jaUMwLxxQqWuzSUtJWZPL3JBf0QNbhOu0mccDAkXYZD0Y9T0Rd_cy43ycJWOuCtcpyMUcPd_CsEkQjTZQVscJUKRU5HldoLKECpyzJUtBj0SoAXJ0B8ea1AFyv_mvXyRPB_Bm-S5txBLEbRRPkMFmxWoKBQqYs7aJB4eLIS9hgmuFzJE__vYmtOoj-lXzEhwTrrGBVkGZ-9eWanig1fhBwVGn3Z6oP2oZ_qVClhWXDCJrMg7wPLJb8Z9tDrgxqOqqqqb2D_umqFIpyOZ9A0ns5zv0FseuUg4U8aoJPmerV5N0SQcdWU_Tkt3QaIvOnn1n9urHNlj0IuckJpEixWemBrZrHPh1k-0cSnqeNTMKgaYMEqft6fxtkrXc5_SqhMmV09dLtuQZOLuOgccvng3mxUxNHbnLUtn4mQ

In [81]:
import pandas as pd
products = pd.DataFrame(data = data, columns = data[0].keys())
products

,product_ID,product_title,price,discount,freeship,product_img_url,product_pg_url,tiki_now,installment_accepted,total_reviews,ratings_avg,cheaper_getRefund,free_gift
0,p58259141,AdĐiện Thoại Samsung Galaxy M31 (6GB/128GB) - ...,4790000,-26%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn//tka.tiki.vn/pixel?data=djAwMem5AAIaFu...,True,Trả góp,1599,4.7,True,
1,p53827214,Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng,2550000,-15%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-oppo-a12-3gb-32gb-hang-chin...,True,,164,4.7,False,
2,p58065346,Điện Thoại Realme C11 (2GB/32GB) - Hàng Chính ...,2190000,-19%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-realme-c11-2gb-32gb-hang-ch...,True,,82,4.8,False,
3,p63839533,Điện Thoại Vsmart Live 4 (6GB/64GB) - Hàng Chí...,3390000,-29%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-vsmart-live-4-hang-chinh-ha...,True,,864,4.7,True,
4,p52717588,AdĐiện Thoại Samsung Galaxy M11 (3GB/32GB) - H...,2790000,-24%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn//tka.tiki.vn/pixel?data=djAwMWuQ_TBlZF...,True,,974,4.7,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,p87054916,HUAWEI P40 Pro+ 5G (8+512G)| Bộ 5 Camera Siêu ...,24990000,-22%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/huawei-p40-pro-5g-8-512g-bo-5-camera-s...,False,,0,0.0,False,
429,p84441714,Điện Thoại Xiaomi Redmi Note 10 (6GB/128GB)- H...,5490000,,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-xiaomi-redmi-note-10-6gb-12...,False,,0,0.0,False,
430,p70840366,Điện thoại để bàn tân cổ điển phím bấm DT08,1950000,-11%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-de-ban-tan-co-dien-phim-bam...,False,,0,0.0,False,
431,p29436154,Điện thoại Yealink T19E2 - Hàng chính Hãng,890000,-41%,Freeship,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-yealink-t19e2-hang-chinh-ha...,False,,4,4.5,False,


In [80]:
products.to_pickle("./result.pkl")
unpickled_result = pd.read_pickle("./result.pkl")
unpickled_result.to_csv("./result.csv", index=False)

In [103]:
products[['total_reviews', 'ratings_avg']].describe()


,total_reviews,ratings_avg
count,433.000000,433.000000
mean,62.720554,3.014088
std,192.176045,2.201832
min,0.000000,0.000000
25%,0.000000,0.000000
50%,4.000000,4.500000
75%,27.000000,4.800000
max,1599.000000,5.000000


In [63]:
print (products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   product_ID            431 non-null    object
 1   product_title         431 non-null    object
 2   price                 431 non-null    int64 
 3   discount              431 non-null    object
 4   freeship              431 non-null    object
 5   product_img_url       431 non-null    object
 6   product_pg_url        431 non-null    object
 7   tiki_now              431 non-null    bool  
 8   installment_accepted  431 non-null    object
 9   total_reviews         431 non-null    object
 10  ratings_avg           431 non-null    object
 11  cheaper_getRefund     431 non-null    bool  
 12  free_gift             431 non-null    object
dtypes: bool(2), int64(1), object(10)
memory usage: 38.0+ KB
None
